In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

from pathlib import Path

import tensorflow.compat.v1 as tf
from matplotlib import pyplot as plt
import numpy as np

from contact_graspnet.dataloading import OrigExampleData
from contact_graspnet.preprocessing import OrigExampleDataPreprocessor, OrigDepth2Points, ZClipper
from contact_graspnet.models import ContactGraspnet
from contact_graspnet.postprocessing import Postprocessor, TopScoreFilter

# from contact_graspnet.orig.contact_graspnet.visualization_utils import visualize_grasps
from contact_graspnet.utils.visualization import mlab_pose_vis

/home/moritz/Documents/ContactGraspnetBenchmark/contact_graspnet/orig/pointnet2/tf_ops/sampling


In [3]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

tf.disable_eager_execution()
physical_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
dataset_path = Path.cwd().parent / "contact_graspnet" / "data" / "orig_test_data"

checkpoint_dir = Path.cwd().parent / "contact_graspnet" / "checkpoints" / "scene_test_2048_bs3_hor_sigma_001"
config_path = checkpoint_dir / "config.yaml"

In [5]:
dataset = OrigExampleData(dataset_path)
sample = dataset[0]

plt.imshow(sample.rgb)

In [7]:
preprocessor = OrigExampleDataPreprocessor(
    OrigDepth2Points(),
    ZClipper(np.array([0.2, 1.8])),
)

model = ContactGraspnet(checkpoint_dir=checkpoint_dir, config_path=config_path, batch_size=1)

postprocessor = Postprocesor(top_score_filter = None)

In [9]:
pointcloud = preprocessor(sample)

network_output = (pred_grasps_cam, scores, contact_pts) = model(pointcloud)

grasps_img = postprocessor(network_output)

Generated 140 grasps


In [11]:
# visualize_grasps(
#     pointcloud, {-1: pred_grasps_cam}, {-1: scores}, plot_opencv_cam=True, pc_colors=preprocessor.intermediate_results["pointcloud_colors"]
# )

In [ ]:
mlab_pose_vis(pointcloud, grasps_img, pc_colors=preprocessor.intermediate_results["pointcloud_colors"])